In [197]:
import numpy as np
import pandas as pd
import xgboost as xgb
from typing import Tuple
from sklearn.multioutput import MultiOutputRegressor

In [198]:
train_df = pd.read_csv('../../data/electricity/train_df.csv')
train_df

,datetime_utc,price_de_lag_336,price_de_avg_24,price_at_lag_24,price_at_avg_24,price_fr_lag_24,price_fr_avg_24,load_de_lag_24,load_de_lag_168,load_at,gen_de,gen_de_lag_168,gen_at,gen_fr,windon_de,solar_de,price_de
0,2022-10-31 23:00:00+00:00,135.71,143.192083,130.59,145.302083,136.77,142.511250,48193.0200,49161.9775,4868.0,49417.49,50407.76,4079.2,40284.0,16063.7425,0.0,80.65
1,2022-11-01 00:00:00+00:00,140.35,141.425000,119.09,144.504167,119.09,140.804167,44770.8675,48136.9350,4378.0,49446.61,49715.33,4027.1,39246.0,16839.0400,0.0,76.68
2,2022-11-01 01:00:00+00:00,139.95,138.880833,116.32,143.657500,116.32,138.950417,43981.9450,47600.7900,4829.0,48568.35,49051.50,3933.9,39366.5,17616.8775,0.0,55.26
3,2022-11-01 02:00:00+00:00,150.30,135.647500,114.60,142.480000,114.60,136.477083,43422.0775,47753.3075,4773.0,48487.32,49083.27,3766.6,37003.5,17984.3850,0.0,37.00
4,2022-11-01 03:00:00+00:00,149.51,132.467917,115.07,141.435417,115.07,134.017500,43581.7450,49360.8175,4833.0,49072.56,50384.18,3585.8,34949.5,18902.0375,0.0,38.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8993,2023-11-10 18:00:00+00:00,120.89,90.798333,112.41,95.982917,112.22,90.937500,64363.1600,62435.3950,7798.0,54701.73,60625.63,7268.6,63410.5,15818.6625,0.0,123.58
8994,2023-11-10 19:00:00+00:00,100.05,91.503333,99.15,96.557083,103.63,91.325000,62330.2025,59991.6850,7289.0,51345.73,60074.36,6770.3,61559.0,15185.0900,0.0,112.93
8995,2023-11-10 20:00:00+00:00,96.84,92.433333,85.54,97.372083,92.50,91.850000,58638.3150,56167.5975,6780.0,47738.81,58340.22,6173.9,59780.5,14189.9350,0.0,105.10
8996,2023-11-10 21:00:00+00:00,93.70,93.263750,84.04,98.202500,84.04,92.680417,55156.1275,52589.0625,6568.0,45213.73,56945.46,5898.0,58930.5,13123.5375,0.0,103.97


In [199]:
test_df = pd.read_csv('../../data/electricity/test_df.csv')
test_df

,datetime_utc,price_de_lag_336,price_de_avg_24,price_at_lag_24,price_at_avg_24,price_fr_lag_24,price_fr_avg_24,load_de_lag_24,load_de_lag_168,load_at,gen_de,gen_de_lag_168,gen_at,gen_fr,windon_de,solar_de,price_de
0,2023-11-10 23:00:00+00:00,68.61,95.015417,65.62,99.890833,64.32,94.422917,48429.1425,45942.1875,5786.0,38133.71,47680.27,5311.9,54197.0,12174.5025,0.0000,87.30
1,2023-11-11 00:00:00+00:00,64.59,96.105417,59.75,100.980000,59.73,95.512917,45261.6750,43090.5175,5565.0,36733.03,47320.98,4997.7,51700.5,11698.9300,0.0000,85.89
2,2023-11-11 01:00:00+00:00,55.00,97.444583,53.74,102.319167,53.74,96.852083,43757.1975,41652.6000,5559.0,34924.21,47398.47,4781.9,50579.0,11689.8875,0.0000,85.88
3,2023-11-11 02:00:00+00:00,60.51,98.896667,48.36,103.771250,48.36,98.304167,43413.8800,40960.8775,5415.0,34693.03,47341.06,4532.1,49535.0,11895.2550,0.0000,83.21
4,2023-11-11 03:00:00+00:00,69.01,100.259583,50.00,105.134167,50.00,99.667083,44354.7750,40866.8275,5445.0,35366.32,48148.71,4500.3,48776.0,12127.9725,0.0000,82.71
5,2023-11-11 04:00:00+00:00,66.47,101.037083,70.19,105.769583,67.76,100.403750,46561.7825,41433.6925,5737.0,36797.09,48837.77,4561.7,48745.0,12433.2300,0.0000,85.44
6,2023-11-11 05:00:00+00:00,79.90,101.125417,99.36,105.225417,85.22,100.448750,50435.6075,42576.4925,6045.0,40130.24,49658.26,4711.0,48975.5,13073.1625,0.0000,86.30
7,2023-11-11 06:00:00+00:00,90.69,100.347917,110.73,104.278333,106.90,99.661250,56524.0175,44376.8950,6453.0,43176.43,50116.83,5267.8,49648.0,13443.2075,233.6975,88.00
8,2023-11-11 07:00:00+00:00,89.50,99.472917,113.00,103.403333,113.00,98.786250,60152.1925,46759.7700,6725.0,46590.94,54396.98,5536.1,51311.5,13536.4125,1913.4275,92.00
9,2023-11-11 08:00:00+00:00,87.01,98.906667,110.26,102.762917,111.05,98.112917,62482.7800,50029.3275,6851.0,49735.18,59324.28,5468.4,53550.0,13538.0750,4785.4625,94.89


In [200]:
X_train_df = pd.read_csv('../../data/electricity/y_train_df.csv')
X_train_df

,datetime_utc,price_de
0,2022-10-31 23:00:00+00:00,80.65
1,2022-11-01 00:00:00+00:00,76.68
2,2022-11-01 01:00:00+00:00,55.26
3,2022-11-01 02:00:00+00:00,37.00
4,2022-11-01 03:00:00+00:00,38.76
...,...,...
8993,2023-11-10 18:00:00+00:00,123.58
8994,2023-11-10 19:00:00+00:00,112.93
8995,2023-11-10 20:00:00+00:00,105.10
8996,2023-11-10 21:00:00+00:00,103.97


In [201]:
X_test_df = pd.read_csv('../../data/electricity/X_test_df.csv')
X_test_df

,datetime_utc,price_de_lag_336,price_de_avg_24,price_at_lag_24,price_at_avg_24,price_fr_lag_24,price_fr_avg_24,load_de_lag_24,load_de_lag_168,load_at,gen_de,gen_de_lag_168,gen_at,gen_fr,windon_de,solar_de
0,2023-11-10 23:00:00+00:00,68.61,95.015417,65.62,99.890833,64.32,94.422917,48429.1425,45942.1875,5786.0,38133.71,47680.27,5311.9,54197.0,12174.5025,0.0000
1,2023-11-11 00:00:00+00:00,64.59,96.105417,59.75,100.980000,59.73,95.512917,45261.6750,43090.5175,5565.0,36733.03,47320.98,4997.7,51700.5,11698.9300,0.0000
2,2023-11-11 01:00:00+00:00,55.00,97.444583,53.74,102.319167,53.74,96.852083,43757.1975,41652.6000,5559.0,34924.21,47398.47,4781.9,50579.0,11689.8875,0.0000
3,2023-11-11 02:00:00+00:00,60.51,98.896667,48.36,103.771250,48.36,98.304167,43413.8800,40960.8775,5415.0,34693.03,47341.06,4532.1,49535.0,11895.2550,0.0000
4,2023-11-11 03:00:00+00:00,69.01,100.259583,50.00,105.134167,50.00,99.667083,44354.7750,40866.8275,5445.0,35366.32,48148.71,4500.3,48776.0,12127.9725,0.0000
5,2023-11-11 04:00:00+00:00,66.47,101.037083,70.19,105.769583,67.76,100.403750,46561.7825,41433.6925,5737.0,36797.09,48837.77,4561.7,48745.0,12433.2300,0.0000
6,2023-11-11 05:00:00+00:00,79.90,101.125417,99.36,105.225417,85.22,100.448750,50435.6075,42576.4925,6045.0,40130.24,49658.26,4711.0,48975.5,13073.1625,0.0000
7,2023-11-11 06:00:00+00:00,90.69,100.347917,110.73,104.278333,106.90,99.661250,56524.0175,44376.8950,6453.0,43176.43,50116.83,5267.8,49648.0,13443.2075,233.6975
8,2023-11-11 07:00:00+00:00,89.50,99.472917,113.00,103.403333,113.00,98.786250,60152.1925,46759.7700,6725.0,46590.94,54396.98,5536.1,51311.5,13536.4125,1913.4275
9,2023-11-11 08:00:00+00:00,87.01,98.906667,110.26,102.762917,111.05,98.112917,62482.7800,50029.3275,6851.0,49735.18,59324.28,5468.4,53550.0,13538.0750,4785.4625


In [202]:
y_train_df = pd.read_csv('../../data/electricity/y_train_df.csv')
y_train_df

,datetime_utc,price_de
0,2022-10-31 23:00:00+00:00,80.65
1,2022-11-01 00:00:00+00:00,76.68
2,2022-11-01 01:00:00+00:00,55.26
3,2022-11-01 02:00:00+00:00,37.00
4,2022-11-01 03:00:00+00:00,38.76
...,...,...
8993,2023-11-10 18:00:00+00:00,123.58
8994,2023-11-10 19:00:00+00:00,112.93
8995,2023-11-10 20:00:00+00:00,105.10
8996,2023-11-10 21:00:00+00:00,103.97


In [203]:
y_test_df = pd.read_csv('../../data/electricity/y_test_df.csv')
y_test_df

,datetime_utc,price_de
0,2023-11-10 23:00:00+00:00,87.30
1,2023-11-11 00:00:00+00:00,85.89
2,2023-11-11 01:00:00+00:00,85.88
3,2023-11-11 02:00:00+00:00,83.21
4,2023-11-11 03:00:00+00:00,82.71
5,2023-11-11 04:00:00+00:00,85.44
6,2023-11-11 05:00:00+00:00,86.30
7,2023-11-11 06:00:00+00:00,88.00
8,2023-11-11 07:00:00+00:00,92.00
9,2023-11-11 08:00:00+00:00,94.89


In [284]:
feature_variable = test_df.drop(columns='datetime_utc').columns
target_variable = 'price_de'
timestemp_col = 'datetime_utc'

hyperparameters = {
    "in_length": 7*24,             # Using 7 days (168 hours) of past observations
    "step_size": 24,               # Sliding the window by 24 steps each time
    "target_sequence_length": 24, # Forecasting 48 hours ahead
    "n_estimators": 20,            # Number of gradient boosted trees
    "max_depth": 6,                # Maximum depth of a tree
    "subsample": 0.5,              # Fraction of samples to be used for fitting each tree
    "min_child_weight": 1,         # Minimum sum of instance weight (hessian) needed in a child
    # "selected_features": [target_variable]  # Features selected for training the model
}

In [204]:
# feature_variable = test_df.drop(columns='datetime_utc').columns
# target_variable = 'price_de'
# timestemp_col = 'datetime_utc'

# hyperparameters = {
#     "in_length": 7*24,             # Using 7 days (168 hours) of past observations
#     "step_size": 24,               # Sliding the window by 24 steps each time
#     "target_sequence_length": 24,  # Forecasting 24 hours ahead
#     "n_estimators": 453,           # Number of gradient boosted trees
#     "max_depth": 3,                # Maximum depth of a tree
#     "learning_rate": 0.015023967309028208,
#     "subsample": 0.8634860353472612,              # Fraction of samples to be used for fitting each tree
#     "min_child_weight": 2,         # Minimum sum of instance weight (hessian) needed in a child
#     'gamma': 0.34786437524253294,
#     # "selected_features": [target_variable]  # Features selected for training the model
# }

In [126]:
Best trial: score 4.980931759422127, params {'n_estimators': 453, 'max_depth': 3, 'learning_rate': 0.015023967309028208, 'subsample': 0.8634860353472612, 'min_child_weight': 2, 'gamma': 0.34786437524253294}

SyntaxError: invalid syntax (545324490.py, line 1)

In [250]:
def get_indices_entire_sequence(data: pd.DataFrame, hyperparameters: dict) -> list:
    """
    Produce all the start and end index positions that are needed to produce
    the sub-sequences for the dataset.

    Args:
        data (pd.DataFrame): Partitioned data set, e.g., training data
        hyperparameters (dict): A dictionary containing the hyperparameters
        
    Return:
        indices: a list of tuples
    """

    window_size = hyperparameters['in_length'] + hyperparameters['target_sequence_length']
    step_size = hyperparameters['step_size']
    stop_position = len(data) - 1

    subseq_first_idx = 0
    subseq_last_idx = window_size

    indices = []

    while subseq_last_idx < stop_position:
        indices.append((subseq_first_idx, subseq_last_idx))
        subseq_first_idx += step_size
        subseq_last_idx += step_size

    return indices

In [251]:
def get_x_y(
        indices: list,
        data: pd.DataFrame,
        target_variable: str,
        feature_variable: list,
        target_sequence_length: int,
        input_seq_len: int,
        # target_col: str = 'price_de'
) -> Tuple[np.array, np.array]:
    
    print ("Preparing data...")
    """
    Obtaining the model inputs and targets (X,Y)
    """
    
    x_data = data[feature_variable].values
    y_data = data[target_variable].values

    for i, idx in enumerate(indices):

        x_instance = x_data[idx[0]:idx[1]]
        y_instance = y_data[idx[0]:idx[1]]

        x = x_instance[0: input_seq_len]
        y = y_instance[input_seq_len:input_seq_len + target_sequence_length]

        assert len(x) == input_seq_len
        assert len(y) == target_sequence_length

        if i == 0:
            X = x.reshape(1, -1)
            Y = y.reshape(1, -1)
        else:
            X = np.concatenate((X, x.reshape(1, -1)), axis=0)
            Y = np.concatenate((Y, y.reshape(1, -1)), axis=0)

    print ("Finished preparing data!")

    return X, Y

In [129]:
training_indices = get_indices_entire_sequence(
        data=train_df,
        hyperparameters=hyperparameters)

x_train, y_train = get_x_y(
        indices=training_indices, 
        data=train_df,
        target_variable=target_variable,
        feature_variable=feature_variable,
        target_sequence_length=hyperparameters["target_sequence_length"],
        input_seq_len=hyperparameters["in_length"]
        )

model = xgb.XGBRegressor(
        n_estimators=hyperparameters["n_estimators"],
        max_depth=hyperparameters["max_depth"],
        subsample=hyperparameters["subsample"],
        learning_rate=hyperparameters["learning_rate"],
        min_child_weight=hyperparameters["min_child_weight"],
        gamma=hyperparameters["gamma"],
        objective="reg:squarederror",
        tree_method="hist"
    )

trained_model = MultiOutputRegressor(model).fit(x_train, y_train)

x_test = train_df[feature_variable].iloc[-hyperparameters["in_length"]:].to_numpy().reshape(1, -1)

y_hat = trained_model.predict(x_test).reshape(-1, 1)
y = y_test_df['price_de'].to_numpy().reshape(-1, 1)

Preparing data...
Finished preparing data!


In [252]:
def smape(y_true, y_pred):
    """
    Calculate the Symmetric Mean Absolute Percentage Error (SMAPE).

    Args:
    y_true (array): True values.
    y_pred (array): Predicted values.

    Returns:
    float: SMAPE score.
    """
    # Avoid division by zero by adding a small epsilon
    epsilon = np.finfo(np.float64).eps
    denominator = np.maximum(np.abs(y_true) + np.abs(y_pred) + epsilon, 0.5 + epsilon)

    # Calculate SMAPE
    smape_value = 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / denominator)
    return smape_value

In [131]:
y = y_test_df['price_de'].to_numpy().reshape(-1, 1)

In [132]:
smape(y, y_hat)

5.166340835403105

### Hyperparameter Tuner

In [280]:
train_val_list = [train_df.iloc[:-8*24], train_df.iloc[-8*24:-8*24+24]]

In [290]:
x_test = train_val_list[0][feature_variable].iloc[-hyperparameters["in_length"]:].to_numpy().reshape(1, -1)

,price_de_lag_336,price_de_avg_24,price_at_lag_24,price_at_avg_24,price_fr_lag_24,price_fr_avg_24,load_de_lag_24,load_de_lag_168,load_at,gen_de,gen_de_lag_168,gen_at,gen_fr,windon_de,solar_de,price_de
8638,109.84,121.345417,112.80,121.550417,112.80,119.191667,50142.1900,49793.9300,5456.0,40724.54,49557.60,4505.4,54701.5,5210.5575,0.0,104.85
8639,123.99,121.973750,90.60,122.132083,92.53,119.692917,46534.4800,46380.6600,5417.0,38588.05,47006.87,3634.4,52974.0,4167.0425,0.0,104.56
8640,115.00,122.282917,91.21,122.370417,93.43,119.760417,43813.0650,44474.8500,5128.0,37243.16,45615.36,3369.0,51892.5,4139.7425,0.0,97.33
8641,107.14,122.600417,89.33,122.485833,92.57,119.151667,42472.1125,42929.7575,5007.0,36606.68,45950.37,3414.6,50446.5,4213.1225,0.0,95.11
8642,101.40,122.932083,88.12,122.545000,90.00,118.067083,41754.5775,42512.3525,4874.0,36690.02,46651.51,3374.9,49315.0,4286.1500,0.0,95.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8801,116.79,53.631250,79.44,62.185833,71.90,51.807083,54640.0175,62010.9675,7656.0,60996.96,52689.68,8200.7,62167.0,28076.5400,0.0,93.84
8802,94.75,55.073333,56.91,63.515833,56.59,53.121667,53419.8650,60341.8425,7115.0,57295.58,50524.15,7887.4,60431.5,26418.1625,0.0,87.76
8803,97.00,56.591667,43.90,64.880417,40.87,54.592500,50400.5225,56669.1775,6625.0,52836.57,47204.90,7245.1,59168.5,24301.5075,0.0,75.95
8804,83.60,58.179167,36.87,66.355417,38.83,55.985833,48097.4250,53357.2275,6356.0,48543.63,44099.05,7248.0,56966.0,21763.3175,0.0,72.27


In [295]:
import optuna

feature_variable = train_val_list[0].drop(columns='datetime_utc').columns
target_variable = 'price_de'
timestemp_col = 'datetime_utc'

y = train_val_list[1][target_variable].to_numpy().reshape(-1, 1)

training_indices = get_indices_entire_sequence(
        data=train_val_list[0],
        hyperparameters=hyperparameters)

x_train, y_train = get_x_y(
        indices=training_indices, 
        data=train_val_list[0],
        target_variable=target_variable,
        feature_variable=feature_variable,
        target_sequence_length=hyperparameters["target_sequence_length"],
        input_seq_len=hyperparameters["in_length"]
        )

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 2, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.1, log=True)
    subsample = trial.suggest_float('subsample', 0.3, 1.0)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 10)
    gamma = trial.suggest_float('gamma', 0, 0.5)
    # colsample_bytree = trial.suggest_uniform('colsample_bytree', 0.5, 1.0)
    reg_alpha = trial.suggest_float('reg_alpha', 0, 1)
    reg_lambda = trial.suggest_float('reg_lambda', 0, 1)

    model = xgb.XGBRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample,
        min_child_weight=min_child_weight,
        gamma=gamma,
        # colsample_bytree=colsample_bytree,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        objective="reg:squarederror",
        tree_method="hist"
    )

    model.fit(x_train, y_train)

    x_test = train_val_list[0][feature_variable].iloc[-hyperparameters["in_length"]:].to_numpy().reshape(1, -1)

    y_hat = model.predict(x_test).reshape(-1, 1)
    
    return smape(y, y_hat)

# def print_smape(trial):
#     print(f'Trial {trial.number}, SMAPE: {trial.value}')


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=15)

print('Best trial: score {}, params {}'.format(study.best_trial.value, study.best_trial.params))

Preparing data...


[I 2023-12-21 13:23:58,240] A new study created in memory with name: no-name-48ff06ef-587a-4862-8483-ac6ca6e0788e


Finished preparing data!


[I 2023-12-21 13:25:56,527] Trial 0 finished with value: 53.149024527982455 and parameters: {'n_estimators': 55, 'max_depth': 9, 'learning_rate': 0.024478740816665127, 'subsample': 0.3966150949762768, 'min_child_weight': 8, 'gamma': 0.20526729046198255, 'reg_alpha': 0.03751279344103531, 'reg_lambda': 0.16035536530772598}. Best is trial 0 with value: 53.149024527982455.
[I 2023-12-21 13:28:52,031] Trial 1 finished with value: 43.112115964105044 and parameters: {'n_estimators': 155, 'max_depth': 5, 'learning_rate': 0.09037350531003088, 'subsample': 0.39676821581205, 'min_child_weight': 7, 'gamma': 0.16494854896981048, 'reg_alpha': 0.556754823029004, 'reg_lambda': 0.3306773743087448}. Best is trial 1 with value: 43.112115964105044.
[I 2023-12-21 13:31:34,414] Trial 2 finished with value: 47.821329345105916 and parameters: {'n_estimators': 90, 'max_depth': 6, 'learning_rate': 0.026010683915489764, 'subsample': 0.5939453699682875, 'min_child_weight': 7, 'gamma': 0.34735240572346837, 'reg_al

Best trial: score 36.24512546409168, params {'n_estimators': 188, 'max_depth': 10, 'learning_rate': 0.04582617138017668, 'subsample': 0.7042668222835616, 'min_child_weight': 3, 'gamma': 0.47446774169252914, 'reg_alpha': 0.04234087357014786, 'reg_lambda': 0.9876203753806796}


In [ ]:
[I 2023-12-21 13:25:56,527] Trial 0 finished with value: 53.149024527982455 and parameters: {'n_estimators': 55, 'max_depth': 9, 'learning_rate': 0.024478740816665127, 'subsample': 0.3966150949762768, 'min_child_weight': 8, 'gamma': 0.20526729046198255, 'reg_alpha': 0.03751279344103531, 'reg_lambda': 0.16035536530772598}. Best is trial 0 with value: 53.149024527982455.
[I 2023-12-21 13:28:52,031] Trial 1 finished with value: 43.112115964105044 and parameters: {'n_estimators': 155, 'max_depth': 5, 'learning_rate': 0.09037350531003088, 'subsample': 0.39676821581205, 'min_child_weight': 7, 'gamma': 0.16494854896981048, 'reg_alpha': 0.556754823029004, 'reg_lambda': 0.3306773743087448}. Best is trial 1 with value: 43.112115964105044.
[I 2023-12-21 13:31:34,414] Trial 2 finished with value: 47.821329345105916 and parameters: {'n_estimators': 90, 'max_depth': 6, 'learning_rate': 0.026010683915489764, 'subsample': 0.5939453699682875, 'min_child_weight': 7, 'gamma': 0.34735240572346837, 'reg_alpha': 0.5343425668412731, 'reg_lambda': 0.9582299014429883}. Best is trial 1 with value: 43.112115964105044.
[I 2023-12-21 13:37:56,755] Trial 3 finished with value: 42.820680499801256 and parameters: {'n_estimators': 151, 'max_depth': 9, 'learning_rate': 0.02300852610483229, 'subsample': 0.5472014655025191, 'min_child_weight': 7, 'gamma': 0.0926913959678633, 'reg_alpha': 0.5607752393883084, 'reg_lambda': 0.6749787099564066}. Best is trial 3 with value: 42.820680499801256.
[I 2023-12-21 13:40:05,184] Trial 4 finished with value: 44.830694165036206 and parameters: {'n_estimators': 110, 'max_depth': 5, 'learning_rate': 0.0813980248623634, 'subsample': 0.4807718918951409, 'min_child_weight': 8, 'gamma': 0.18273216801004077, 'reg_alpha': 0.407890059303027, 'reg_lambda': 0.33113012760240135}. Best is trial 3 with value: 42.820680499801256.
[I 2023-12-21 13:46:10,820] Trial 5 finished with value: 54.94856779842467 and parameters: {'n_estimators': 112, 'max_depth': 9, 'learning_rate': 0.011523785310572155, 'subsample': 0.8323748902582624, 'min_child_weight': 8, 'gamma': 0.35643007626887385, 'reg_alpha': 0.2484409912230805, 'reg_lambda': 0.41693531186529176}. Best is trial 3 with value: 42.820680499801256.
[I 2023-12-21 13:47:10,196] Trial 6 finished with value: 67.24181649980115 and parameters: {'n_estimators': 78, 'max_depth': 3, 'learning_rate': 0.007068165041330318, 'subsample': 0.9528099973827868, 'min_child_weight': 5, 'gamma': 0.45711817667979965, 'reg_alpha': 0.7193103275350518, 'reg_lambda': 0.5003526133620493}. Best is trial 3 with value: 42.820680499801256.
[I 2023-12-21 13:54:02,000] Trial 7 finished with value: 42.02858497117343 and parameters: {'n_estimators': 124, 'max_depth': 9, 'learning_rate': 0.040558052283461946, 'subsample': 0.7044875606686882, 'min_child_weight': 5, 'gamma': 0.3875758433294676, 'reg_alpha': 0.09845583265192903, 'reg_lambda': 0.7897072707112114}. Best is trial 7 with value: 42.02858497117343.
[I 2023-12-21 13:58:36,189] Trial 8 finished with value: 63.10362833214771 and parameters: {'n_estimators': 121, 'max_depth': 6, 'learning_rate': 0.006117862578870449, 'subsample': 0.31633258512393125, 'min_child_weight': 1, 'gamma': 0.07033366880260905, 'reg_alpha': 0.8199140445914276, 'reg_lambda': 0.7028161099122262}. Best is trial 7 with value: 42.02858497117343.
[I 2023-12-21 14:02:46,799] Trial 9 finished with value: 48.39924205287904 and parameters: {'n_estimators': 168, 'max_depth': 8, 'learning_rate': 0.013627024423404752, 'subsample': 0.30185012219187674, 'min_child_weight': 10, 'gamma': 0.029968977367124006, 'reg_alpha': 0.9800779868454723, 'reg_lambda': 0.44914182072791986}. Best is trial 7 with value: 42.02858497117343.
[I 2023-12-21 14:15:49,850] Trial 10 finished with value: 39.81650278942883 and parameters: {'n_estimators': 179, 'max_depth': 10, 'learning_rate': 0.04648892720045159, 'subsample': 0.7627876056327054, 'min_child_weight': 3, 'gamma': 0.49299772795066893, 'reg_alpha': 0.11555613777218268, 'reg_lambda': 0.983417349427868}. Best is trial 10 with value: 39.81650278942883.
[I 2023-12-21 14:29:20,409] Trial 11 finished with value: 36.24512546409168 and parameters: {'n_estimators': 188, 'max_depth': 10, 'learning_rate': 0.04582617138017668, 'subsample': 0.7042668222835616, 'min_child_weight': 3, 'gamma': 0.47446774169252914, 'reg_alpha': 0.04234087357014786, 'reg_lambda': 0.9876203753806796}. Best is trial 11 with value: 36.24512546409168.
[I 2023-12-21 14:44:40,120] Trial 12 finished with value: 36.775936943049 and parameters: {'n_estimators': 197, 'max_depth': 10, 'learning_rate': 0.04830521951388083, 'subsample': 0.7212327514807602, 'min_child_weight': 2, 'gamma': 0.45765713460983265, 'reg_alpha': 0.008837581121710208, 'reg_lambda': 0.9854276719758106}. Best is trial 11 with value: 36.24512546409168.
[I 2023-12-21 14:59:10,562] Trial 13 finished with value: 37.88235518812936 and parameters: {'n_estimators': 199, 'max_depth': 10, 'learning_rate': 0.052066122946410255, 'subsample': 0.6613532905219177, 'min_child_weight': 2, 'gamma': 0.41539699232343785, 'reg_alpha': 0.018309107986919305, 'reg_lambda': 0.992036798588328}. Best is trial 11 with value: 36.24512546409168.
[I 2023-12-21 15:08:19,014] Trial 14 finished with value: 39.49477100204656 and parameters: {'n_estimators': 189, 'max_depth': 7, 'learning_rate': 0.059969328061803974, 'subsample': 0.768995471481744, 'min_child_weight': 3, 'gamma': 0.4968958720402855, 'reg_alpha': 0.23870438010176234, 'reg_lambda': 0.8362320541856051}. Best is trial 11 with value: 36.24512546409168.
Best trial: score 36.24512546409168, params {'n_estimators': 188, 'max_depth': 10, 'learning_rate': 0.04582617138017668, 'subsample': 0.7042668222835616, 'min_child_weight': 3, 'gamma': 0.47446774169252914, 'reg_alpha': 0.04234087357014786, 'reg_lambda': 0.9876203753806796}


### Sliding window train set generator

In [297]:
index_cutoffs = [24*i for i in range(7, -1, -1)]
index_cutoffs

[168, 144, 120, 96, 72, 48, 24, 0]

In [374]:
train_df_list = [train_df.iloc[:-idx] if idx != 0 else train_df for idx in index_cutoffs]
index_ceiling = [x.index.stop for x in train_df_list]
test_df_list = [train_df['price_de'].iloc[idx:idx+24] if idx!=8998 else test_df['price_de'] for idx in index_ceiling]

In [378]:
[x.index.start for x in train_df_list]

[0, 0, 0, 0, 0, 0, 0, 0]

In [375]:
[x.index.stop for x in train_df_list]

[8830, 8854, 8878, 8902, 8926, 8950, 8974, 8998]

In [376]:
[x.index for x in test_df_list]

[RangeIndex(start=8830, stop=8854, step=1),
 RangeIndex(start=8854, stop=8878, step=1),
 RangeIndex(start=8878, stop=8902, step=1),
 RangeIndex(start=8902, stop=8926, step=1),
 RangeIndex(start=8926, stop=8950, step=1),
 RangeIndex(start=8950, stop=8974, step=1),
 RangeIndex(start=8974, stop=8998, step=1),
 RangeIndex(start=0, stop=24, step=1)]

In [380]:
test_df_list = [train_df['price_de'].iloc[idx:idx+hyperparameters['step_size']] if idx!=index_ceiling[-1] else test_df['price_de'] for idx in index_ceiling]

In [385]:
test_df_list[0].to_numpy().reshape(-1, 1)

array([[45.72],
       [32.06],
       [30.  ],
       [22.01],
       [19.19],
       [15.4 ],
       [16.13],
       [25.08],
       [25.85],
       [20.  ],
       [13.95],
       [15.2 ],
       [20.  ],
       [21.14],
       [30.79],
       [42.69],
       [40.  ],
       [48.76],
       [40.  ],
       [29.61],
       [12.82],
       [10.4 ],
       [11.73],
       [ 7.05]])

In [366]:
test_df_list[-2]

Series([], Name: price_de, dtype: float64)

In [296]:
feature_variable = test_df.drop(columns='datetime_utc').columns
target_variable = 'price_de'
timestemp_col = 'datetime_utc'

hyperparameters = {
    "in_length": 7*24,             # Using 7 days (168 hours) of past observations
    "step_size": 24,               # Sliding the window by 24 steps each time
    "target_sequence_length": 24,  # Forecasting 24 hours ahead
    "n_estimators": 188,           # Number of gradient boosted trees
    "max_depth": 10,                # Maximum depth of a tree
    "learning_rate": 0.04582617138017668,
    "subsample": 0.7042668222835616,              # Fraction of samples to be used for fitting each tree
    "min_child_weight": 3,         # Minimum sum of instance weight (hessian) needed in a child
    'gamma': 0.47446774169252914,
    'reg_alpha': 0.04234087357014786,
    'reg_lambda': 0.9876203753806796
    # "selected_features": [target_variable]  # Features selected for training the model
}

In [300]:
# empty y_hat dataframe
y_hat_df = pd.concat([train_df.iloc[-168:][['datetime_utc']], test_df[['datetime_utc']]], axis=0).reset_index(drop=True)
y_hat = np.empty((0, x_test.shape[0]))

for train_df_temp in [train_df_list[0]]:

        training_indices = get_indices_entire_sequence(
                data=train_df_temp,
                hyperparameters=hyperparameters)

        x_train, y_train = get_x_y(
                indices=training_indices, 
                data=train_df_temp,
                target_variable=target_variable,
                feature_variable=feature_variable,
                target_sequence_length=hyperparameters["target_sequence_length"],
                input_seq_len=hyperparameters["in_length"]
                )

        model = xgb.XGBRegressor(
                n_estimators=hyperparameters["n_estimators"],
                max_depth=hyperparameters["max_depth"],
                subsample=hyperparameters["subsample"],
                learning_rate=hyperparameters["learning_rate"],
                min_child_weight=hyperparameters["min_child_weight"],
                gamma=hyperparameters["gamma"],
                reg_alpha=hyperparameters["reg_alpha"],
                reg_lambda=hyperparameters["reg_lambda"],
                objective="reg:squarederror",
                tree_method="hist"
        )

        trained_model = MultiOutputRegressor(model).fit(x_train, y_train)

#         x_test = train_df_temp[feature_variable].iloc[-hyperparameters["in_length"]:].to_numpy().reshape(1, -1)

#         y_hat = np.vstack((y_hat, trained_model.predict(x_test).reshape(-1, 1)))


# y = y_test_df['price_de'].to_numpy().reshape(-1, 1)

Preparing data...
Finished preparing data!


In [322]:
x_test = train_df[feature_variable].iloc[-hyperparameters["in_length"]:].to_numpy().reshape(1, -1)

y_hat = trained_model.predict(x_test).reshape(-1, 1)

In [323]:
smape(y_hat, y_test_df['price_de'].to_numpy().reshape(-1, 1))

9.4731683274814

In [350]:
train_val_list_test = [train_df.iloc[:-3*24], train_df.iloc[-3*24:-3*24+24]]

In [351]:
x_test = train_val_list_test[0][feature_variable].iloc[-hyperparameters["in_length"]:].to_numpy().reshape(1, -1)

y_hat = trained_model.predict(x_test).reshape(-1, 1)

In [352]:
smape(y_hat , train_val_list_test[1][target_variable].to_numpy().reshape(-1, 1))

9.541827304370846

In [194]:
y_hat_copy = y_hat

In [313]:
train_df_temp

,datetime_utc,price_de_lag_336,price_de_avg_24,price_at_lag_24,price_at_avg_24,price_fr_lag_24,price_fr_avg_24,load_de_lag_24,load_de_lag_168,load_at,gen_de,gen_de_lag_168,gen_at,gen_fr,windon_de,solar_de,price_de
0,2022-10-31 23:00:00+00:00,135.71,143.192083,130.59,145.302083,136.77,142.511250,48193.0200,49161.9775,4868.0,49417.49,50407.76,4079.2,40284.0,16063.7425,0.0,80.65
1,2022-11-01 00:00:00+00:00,140.35,141.425000,119.09,144.504167,119.09,140.804167,44770.8675,48136.9350,4378.0,49446.61,49715.33,4027.1,39246.0,16839.0400,0.0,76.68
2,2022-11-01 01:00:00+00:00,139.95,138.880833,116.32,143.657500,116.32,138.950417,43981.9450,47600.7900,4829.0,48568.35,49051.50,3933.9,39366.5,17616.8775,0.0,55.26
3,2022-11-01 02:00:00+00:00,150.30,135.647500,114.60,142.480000,114.60,136.477083,43422.0775,47753.3075,4773.0,48487.32,49083.27,3766.6,37003.5,17984.3850,0.0,37.00
4,2022-11-01 03:00:00+00:00,149.51,132.467917,115.07,141.435417,115.07,134.017500,43581.7450,49360.8175,4833.0,49072.56,50384.18,3585.8,34949.5,18902.0375,0.0,38.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8825,2023-11-03 18:00:00+00:00,101.94,60.933750,97.19,63.049167,96.55,61.328750,62388.3700,59901.7900,7312.0,60625.63,51657.31,7450.4,60633.0,26007.1175,0.0,88.23
8826,2023-11-03 19:00:00+00:00,82.86,60.155000,88.83,62.428333,88.14,60.582917,60591.5200,57990.4625,6811.0,60074.36,49932.74,6842.7,59365.0,27109.9175,0.0,69.07
8827,2023-11-03 20:00:00+00:00,78.51,59.204167,76.65,61.460417,76.17,59.642917,56832.7400,54501.0425,6360.0,58340.22,47235.45,6123.4,57190.5,27751.8150,0.0,53.13
8828,2023-11-03 21:00:00+00:00,56.44,58.090000,72.27,60.346250,72.27,58.528750,53339.9050,51522.8225,6137.0,56945.46,43269.00,6054.1,55806.0,27833.9575,0.0,45.53


In [314]:
train_df_temp[feature_variable].iloc[-hyperparameters["in_length"]:]

,price_de_lag_336,price_de_avg_24,price_at_lag_24,price_at_avg_24,price_fr_lag_24,price_fr_avg_24,load_de_lag_24,load_de_lag_168,load_at,gen_de,gen_de_lag_168,gen_at,gen_fr,windon_de,solar_de,price_de
8662,-0.81,118.350000,104.85,117.939167,104.85,100.709583,49658.6375,48720.5125,5382.0,37719.74,45699.97,5380.9,54138.0,11161.3500,0.0,93.70
8663,-0.09,117.518333,104.56,117.107500,104.56,99.877917,46061.6100,45265.9650,5472.0,36650.76,43851.36,5490.2,51054.0,11440.7075,0.0,84.60
8664,-1.87,116.321667,96.93,115.927500,95.05,98.776250,44089.3750,43085.1050,5147.0,35973.77,42719.17,5244.1,49593.0,11766.5600,0.0,68.61
8665,-1.49,115.050000,92.10,114.781250,77.96,98.219167,42723.2475,41535.9350,5098.0,35403.01,41371.16,5086.1,47760.0,11928.7875,0.0,64.59
8666,-1.72,113.383333,89.54,113.342083,63.97,97.845417,42469.8950,40702.7450,4974.0,35079.06,40396.56,4866.1,47540.5,12362.0200,0.0,55.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8825,101.94,60.933750,97.19,63.049167,96.55,61.328750,62388.3700,59901.7900,7312.0,60625.63,51657.31,7450.4,60633.0,26007.1175,0.0,88.23
8826,82.86,60.155000,88.83,62.428333,88.14,60.582917,60591.5200,57990.4625,6811.0,60074.36,49932.74,6842.7,59365.0,27109.9175,0.0,69.07
8827,78.51,59.204167,76.65,61.460417,76.17,59.642917,56832.7400,54501.0425,6360.0,58340.22,47235.45,6123.4,57190.5,27751.8150,0.0,53.13
8828,56.44,58.090000,72.27,60.346250,72.27,58.528750,53339.9050,51522.8225,6137.0,56945.46,43269.00,6054.1,55806.0,27833.9575,0.0,45.53


In [181]:
y = pd.concat([train_df.iloc[-168:][['price_de']], test_df[['price_de']]], axis=0).reset_index(drop=True).to_numpy().reshape(-1, 1)
y

array([[4.5720e+01],
       [3.2060e+01],
       [3.0000e+01],
       [2.2010e+01],
       [1.9190e+01],
       [1.5400e+01],
       [1.6130e+01],
       [2.5080e+01],
       [2.5850e+01],
       [2.0000e+01],
       [1.3950e+01],
       [1.5200e+01],
       [2.0000e+01],
       [2.1140e+01],
       [3.0790e+01],
       [4.2690e+01],
       [4.0000e+01],
       [4.8760e+01],
       [4.0000e+01],
       [2.9610e+01],
       [1.2820e+01],
       [1.0400e+01],
       [1.1730e+01],
       [7.0500e+00],
       [8.7100e+00],
       [6.5900e+00],
       [5.1800e+00],
       [2.9800e+00],
       [4.2400e+00],
       [7.9900e+00],
       [7.2900e+00],
       [1.1290e+01],
       [1.5580e+01],
       [2.4020e+01],
       [4.7470e+01],
       [4.8460e+01],
       [4.4280e+01],
       [2.9700e+01],
       [3.8150e+01],
       [5.7180e+01],
       [6.3090e+01],
       [7.4950e+01],
       [7.2670e+01],
       [5.9170e+01],
       [4.6480e+01],
       [2.0850e+01],
       [1.5840e+01],
       [5.710

In [210]:
y = train_df.iloc[-168:-144][['price_de']]

In [211]:
y-y_hat

,price_de
8830,38.548465
8831,21.380208
8832,16.875973
8833,6.506267
8834,-5.348193
8835,-31.082300
8836,-32.105256
8837,-21.408941
8838,-19.613947
8839,-22.615269


In [212]:
smape(y, y_hat)

price_de    79.974146
dtype: float64

In [187]:
smape(y[:24], y_hat[:24])

79.97414592730652

In [190]:
trained_model.predict(x_test).reshape(-1, 1)

array([[ 88.36422 ],
       [ 86.267204],
       [ 82.29413 ],
       [ 83.79184 ],
       [ 86.27128 ],
       [ 87.44459 ],
       [100.18474 ],
       [ 96.55963 ],
       [ 89.69971 ],
       [ 92.620155],
       [ 89.22869 ],
       [ 82.52214 ],
       [ 80.813805],
       [ 79.70977 ],
       [ 89.48583 ],
       [101.34096 ],
       [103.530426],
       [115.991104],
       [123.20195 ],
       [121.821144],
       [109.832596],
       [ 99.29693 ],
       [ 95.66245 ],
       [ 82.417435]], dtype=float32)

In [218]:
y_hat

array([[  7.17153549],
       [ 10.6797924 ],
       [ 13.12402725],
       [ 15.50373268],
       [ 24.53819275],
       [ 46.4822998 ],
       [ 48.2352562 ],
       [ 46.48894119],
       [ 45.4639473 ],
       [ 42.61526871],
       [ 42.34431076],
       [ 31.92834854],
       [ 26.54858398],
       [ 20.35617828],
       [ 23.73907471],
       [ 47.33136368],
       [ 60.86796951],
       [ 87.64021301],
       [102.43487549],
       [ 98.46813965],
       [ 91.55648804],
       [ 80.09924316],
       [ 75.68221283],
       [ 71.5105896 ]])

In [220]:
trained_model.predict(x_test).reshape(-1, 1)

array([[  7.1715355],
       [ 10.679792 ],
       [ 13.124027 ],
       [ 15.503733 ],
       [ 24.538193 ],
       [ 46.4823   ],
       [ 48.235256 ],
       [ 46.48894  ],
       [ 45.463947 ],
       [ 42.61527  ],
       [ 42.34431  ],
       [ 31.928349 ],
       [ 26.548584 ],
       [ 20.356178 ],
       [ 23.739075 ],
       [ 47.331364 ],
       [ 60.86797  ],
       [ 87.64021  ],
       [102.434875 ],
       [ 98.46814  ],
       [ 91.55649  ],
       [ 80.09924  ],
       [ 75.68221  ],
       [ 71.51059  ]], dtype=float32)

In [260]:
x_test = train_df[feature_variable].iloc[-hyperparameters["in_length"]:].to_numpy().reshape(1, -1)
trained_model.predict(x_test).reshape(-1, 1) - y_test_df['price_de'].to_numpy().reshape(-1, 1)

array([[  7.47339172],
       [ -0.49110474],
       [-13.63984192],
       [ -7.67430969],
       [ -8.97383209],
       [ 18.58700043],
       [ 25.31383057],
       [ 17.65796661],
       [ -7.01966858],
       [-20.51519836],
       [-18.83818176],
       [-26.90264008],
       [ -7.78542847],
       [-38.23216324],
       [-54.85690689],
       [-72.00191414],
       [-23.13410065],
       [-27.93069885],
       [ 11.56086182],
       [-14.97635529],
       [ -4.51618744],
       [ -7.55801971],
       [ 14.81930267],
       [ 17.28327545]])

In [243]:
y_test_df

,datetime_utc,price_de
0,2023-11-10 23:00:00+00:00,87.30
1,2023-11-11 00:00:00+00:00,85.89
2,2023-11-11 01:00:00+00:00,85.88
3,2023-11-11 02:00:00+00:00,83.21
4,2023-11-11 03:00:00+00:00,82.71
5,2023-11-11 04:00:00+00:00,85.44
6,2023-11-11 05:00:00+00:00,86.30
7,2023-11-11 06:00:00+00:00,88.00
8,2023-11-11 07:00:00+00:00,92.00
9,2023-11-11 08:00:00+00:00,94.89


In [312]:
x_test = train_df_temp[feature_variable].iloc[-hyperparameters["in_length"]:].to_numpy().reshape(1, -1)
trained_model.predict(x_test).reshape(-1, 1) - y_train_df[8830:8830+24]['price_de'].to_numpy().reshape(-1, 1)

array([[-30.90304825],
       [-15.31338936],
       [ -5.97303009],
       [  0.56944298],
       [ -2.3723616 ],
       [ 13.29929695],
       [ 19.76777756],
       [  5.6327533 ],
       [ 13.70812836],
       [ 34.62963486],
       [ 40.10871201],
       [ 34.69966583],
       [ 12.67552567],
       [  6.86914955],
       [ -6.14070801],
       [ -2.98986954],
       [ 17.54048157],
       [  0.29147934],
       [ 60.17041016],
       [ 78.47973694],
       [ 88.00436371],
       [ 71.00625763],
       [ 61.15839722],
       [ 61.99478455]])

In [261]:
y_train_df[8830:8830+24]

,datetime_utc,price_de
8830,2023-11-03 23:00:00+00:00,45.72
8831,2023-11-04 00:00:00+00:00,32.06
8832,2023-11-04 01:00:00+00:00,30.00
8833,2023-11-04 02:00:00+00:00,22.01
8834,2023-11-04 03:00:00+00:00,19.19
8835,2023-11-04 04:00:00+00:00,15.40
8836,2023-11-04 05:00:00+00:00,16.13
8837,2023-11-04 06:00:00+00:00,25.08
8838,2023-11-04 07:00:00+00:00,25.85
8839,2023-11-04 08:00:00+00:00,20.00


In [245]:
train_df_temp[feature_variable].iloc[-hyperparameters["in_length"]:]

,price_de_lag_336,price_de_avg_24,price_at_lag_24,price_at_avg_24,price_fr_lag_24,price_fr_avg_24,load_de_lag_24,load_de_lag_168,load_at,gen_de,gen_de_lag_168,gen_at,gen_fr,windon_de,solar_de,price_de
8662,-0.81,118.350000,104.85,117.939167,104.85,100.709583,49658.6375,48720.5125,5382.0,37719.74,45699.97,5380.9,54138.0,11161.3500,0.0,93.70
8663,-0.09,117.518333,104.56,117.107500,104.56,99.877917,46061.6100,45265.9650,5472.0,36650.76,43851.36,5490.2,51054.0,11440.7075,0.0,84.60
8664,-1.87,116.321667,96.93,115.927500,95.05,98.776250,44089.3750,43085.1050,5147.0,35973.77,42719.17,5244.1,49593.0,11766.5600,0.0,68.61
8665,-1.49,115.050000,92.10,114.781250,77.96,98.219167,42723.2475,41535.9350,5098.0,35403.01,41371.16,5086.1,47760.0,11928.7875,0.0,64.59
8666,-1.72,113.383333,89.54,113.342083,63.97,97.845417,42469.8950,40702.7450,4974.0,35079.06,40396.56,4866.1,47540.5,12362.0200,0.0,55.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8825,101.94,60.933750,97.19,63.049167,96.55,61.328750,62388.3700,59901.7900,7312.0,60625.63,51657.31,7450.4,60633.0,26007.1175,0.0,88.23
8826,82.86,60.155000,88.83,62.428333,88.14,60.582917,60591.5200,57990.4625,6811.0,60074.36,49932.74,6842.7,59365.0,27109.9175,0.0,69.07
8827,78.51,59.204167,76.65,61.460417,76.17,59.642917,56832.7400,54501.0425,6360.0,58340.22,47235.45,6123.4,57190.5,27751.8150,0.0,53.13
8828,56.44,58.090000,72.27,60.346250,72.27,58.528750,53339.9050,51522.8225,6137.0,56945.46,43269.00,6054.1,55806.0,27833.9575,0.0,45.53


In [246]:
train_df[feature_variable].iloc[-hyperparameters["in_length"]:]

,price_de_lag_336,price_de_avg_24,price_at_lag_24,price_at_avg_24,price_fr_lag_24,price_fr_avg_24,load_de_lag_24,load_de_lag_168,load_at,gen_de,gen_de_lag_168,gen_at,gen_fr,windon_de,solar_de,price_de
8830,37.27,57.845833,15.17,60.102083,15.17,58.378333,46580.1925,44820.6800,5368.0,47680.27,36650.76,6015.2,53216.5,22518.3650,0.0,45.72
8831,39.74,58.703750,11.47,60.960000,11.47,59.236250,45325.7825,42251.9325,5131.0,47320.98,35973.77,5788.3,51448.5,23020.7750,0.0,32.06
8832,33.95,59.537083,10.00,61.793333,10.00,60.069583,43936.4350,40700.3325,5137.0,47398.47,35403.01,5646.7,48367.0,23561.6125,0.0,30.00
8833,37.15,60.215417,5.73,62.471667,5.73,60.747917,43577.4950,39854.2300,5145.0,47341.06,35079.06,5487.2,46876.5,23908.4950,0.0,22.01
8834,35.14,60.609583,9.73,62.865833,9.73,61.142083,44400.7025,40046.5975,5408.0,48148.71,35556.24,5410.5,46899.5,24023.1625,0.0,19.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8993,120.89,90.798333,112.41,95.982917,112.22,90.937500,64363.1600,62435.3950,7798.0,54701.73,60625.63,7268.6,63410.5,15818.6625,0.0,123.58
8994,100.05,91.503333,99.15,96.557083,103.63,91.325000,62330.2025,59991.6850,7289.0,51345.73,60074.36,6770.3,61559.0,15185.0900,0.0,112.93
8995,96.84,92.433333,85.54,97.372083,92.50,91.850000,58638.3150,56167.5975,6780.0,47738.81,58340.22,6173.9,59780.5,14189.9350,0.0,105.10
8996,93.70,93.263750,84.04,98.202500,84.04,92.680417,55156.1275,52589.0625,6568.0,45213.73,56945.46,5898.0,58930.5,13123.5375,0.0,103.97
